To begin, you need to have processed your IMAs through the Step 1 notebook. You are now going to align all of the individual diff files from those IMA files to an outside catalog and then use TweakBack to apply the distortions back onto the individual diff files. This will allow us to mosaic several exposures of DASH data in notebook Step 3. 


Table of contents: 

1. <a href='#align_cat'> Align reads to catalog <br> </a>
2. <a href='#align_both'> Align reads to each other, then to a catalog <br> </a>

In [ ]:
from astroquery.mast import Observations
from astropy.io import fits 
from astropy.table import Table
import matplotlib.pyplot as plt 
import numpy as np

from astropy.io import ascii

from glob import glob
from drizzlepac import astrodrizzle

import os

%matplotlib notebook 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
tmp_path = "/Users/cmartlin/Desktop/DASH/wfc3_dash/wfc3_dash"
if module_path not in sys.path:
    sys.path.append(module_path)
if tmp_path not in sys.path:
    sys.path.append(tmp_path)

from reduce_dash import DashData

### 1. Align reads to COSMOS

#### Align images.
Parameters ref_catalog and ref_image denote the reference catalog and reference image, respectively.  
Note: Must be connected to STScI network in order to align to Gaia (specifically to use updatewcs function on the input images for TweakReg). To not use this function, set parameter updatewcsfn to False.

NB: If an error arises, try lowering the threshold.

In [ ]:
from utils import get_IDCtable

### Gather the data you will be using first: 

In [ ]:
obsTable = Observations.query_criteria(proposal_id=['14114'])

product_list = Observations.get_product_list(obsTable)
BM = (product_list['productSubGroupDescription']  == 'IMA') 
product_list = product_list[BM]

myID = product_list['obsID'][0:1]

download = Observations.download_products(myID,mrp_only=False,productSubGroupDescription=['IMA','FLT'])

### Create a DASH object from your file as we did in the first notebook: 

In [ ]:
localpathtofile = download['Local Path'][0][:-8]

myDash = DashData(localpathtofile+'ima.fits', flt_file_name=localpathtofile+'flt.fits')

In [ ]:
get_IDCtable('mastDownload/HST/icxe01tyq/icxe01tyq_ima.fits')

In [ ]:
import urllib.request
import os

if not os.path.exists('catalogs'):
    os.mkdir('catalogs')

print('Beginning catalog download...')

url = 'https://irsa.ipac.caltech.edu/data/COSMOS/tables/photometry/cosmos_acs_iphot_200709.tbl'
urllib.request.urlretrieve(url, 'catalogs/refcat.tbl')

Make a catalog consisting of only the RA and DEC from the COSMOS catalog.

In [ ]:
os.system("awk '{ print $35, $36 }' catalogs/refcat.tbl > catalogs/cosmos_ra_dec.tbl")

Remove header of catalog.

In [ ]:
n = 124
nfirstlines = []

with open('catalogs/cosmos_ra_dec.tbl') as f, open("catalogs/temp_cat.dat", "w") as out:
    for x in range(n):
        nfirstlines.append(next(f))
    for line in f:
        out.write(line)

os.remove('catalogs/cosmos_ra_dec.tbl')
os.rename("catalogs/temp_cat.dat", 'catalogs/cosmos_ra_dec.tbl')

Align to COSMOS catalog.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='catalogs/cosmos_ra_dec.tbl', 
             cat_file='catalogs/diff_catfile.cat',
             threshold=40., searchrad=60., 
             cw=3.5, 
             updatehdr=False, 
             updatewcs=False, 
             astrodriz=False)

Inspect the shifts file to see if method used produced sufficient results. Notice that some diff files could not sufficiently line up with Gaia. It is up to the user to determine whether this is sufficient or not.

In [ ]:
print(open('shifts/shifts_icxe01tyq.txt').read())

Update header and plot aligned science images.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='catalogs/cosmos_ra_dec.tbl', 
             cat_file='catalogs/diff_catfile.cat',
             threshold=40., searchrad=60., cw=3.5, wcsname='DASH2', move_files=True)

In [ ]:
sci = fits.getdata('icxe01tyq_drz_sci.fits')

fig = plt.figure(figsize=(10, 10))

plt.imshow(sci, vmin=-0.05, vmax=0.4, cmap='Greys', origin='lower')

# 2. Align the reads to each other, then align the final read to a catalog

Will most likely give better results, especially if not enough sources are found using catalog method alone.


### Align sources to each other using TweakReg.

In [ ]:
myDash.align(astrodriz=False)

### Align updated images to COSMOS.

In [ ]:
myDash.align(align_method='CATALOG', 
             ref_catalog='catalogs/cosmos_ra_dec.tbl', 
             cat_file='catalogs/diff_catfile.cat',
             threshold=40., searchrad=60., cw=3.5, wcsname='DASH3')

Inspect the shifts file to see if method used produced sufficient results. Then plot final drizzled image.

In [ ]:
print(open('shifts_icxe01tyq.txt').read())

In [ ]:
sci = fits.getdata('icxe01tyq_drz_sci.fits')

fig = plt.figure(figsize=(20, 20))

plt.imshow(sci, vmin=-0.05, vmax=0.4, cmap='Greys', origin='lower')

Once you have a successful fit then you can run TweakBack to apply the WCS changes to the original diff files you created in step 1. Step 3 is next and with the distortion free diff files you can use Astrodrizzle to create a mosaic of your diff IMA files. 